# Denoise Seismic Waveform for Earthquakes

In [ ]:
import h5py
import numpy as np

from obspy import UTCDateTime
from obspy.taup import TauPyModel
from obspy.clients.fdsn import Client
from obspy.geodetics import locations2degrees

import matplotlib
from matplotlib import pyplot as plt
matplotlib.rcParams.update({'font.size': 12})
%matplotlib inline

## Step 0. Prepare 3-component earthquake waveforms


First, to find earthquakes of interest.

We use the ISC client to get a earthquake catalog

In [ ]:

t1 = UTCDateTime("2002-02-01")
t2 = UTCDateTime("2002-02-02")
cat = Client("ISC").get_events(starttime=t1,
                                endtime=t2,
                                minmagnitude=5.0,
                                maxmagnitude=6.5,
                                mindepth=100.0,
                                maxdepth=600.0)
print(cat)

---
Let's start with one of the earthquakes, and get its location and time.

In [ ]:
ev = cat[0]
ev_t = ev.origins[0].time
evlo = ev.origins[0].longitude
evla = ev.origins[0].latitude
evdp = ev.origins[0].depth / 1000
print(f'Event time: {ev_t}, Event location: {evlo:.2f}°, {evla:.2f}°, {evdp:.0f} km')

---
Then, we need to download the data. What stations do you like?

In [ ]:
network = 'IU'
station_list = [ "ANMO", "CTAO", "GNI", "INCN", "KEV", "MAJO", "HNR", "KNTN", "NWAO", "POHA", "PTCN", "SFJD", "SNZO", "TATO", "ULN"]
model = TauPyModel(model="iasp91")


Some parameters for pre-processing the data. 

The key is to trim the 3-component traces to 1500 samples.

In [ ]:
dt = 0.1  # sampling interval
npts = 1500  # number of samples of each trace
pre_len_ratio = 0.5  # ratio of the noise window length to the total trace length
preP = int(pre_len_ratio * npts * dt)  # number of samples of the noise window before the P arrival
aftP = npts * dt - preP  # number of samples of the noise window after the P arrival


Download data for the stations you defined and do the pre-processing 

In [ ]:
all_data = []
### Process each station
for station in station_list:
    ### Get station coordinates 
    inv = Client("IRIS").get_stations(network=network, station=station, level='channel')
    stla = inv[0][0].latitude
    stlo = inv[0][0].longitude

    ### Calculate the distance and the theoretical arrival time
    dist_deg = locations2degrees(evla, evlo, stla, stlo)
    arrivals = model.get_travel_times(source_depth_in_km=evdp, 
                                      distance_in_degree=dist_deg)
    tp = UTCDateTime(ev_t + arrivals[0].time)
    print(f"Station {station} is located at {stla:.2f}°, {stlo:.2f}°, distance to the event is {dist_deg:.2f}°")

    try:
        ### Download the data
        st = Client("IRIS").get_waveforms(network=network, 
                                          station=station, 
                                          location='00', 
                                          channel='BH?',
                                          starttime=tp - preP, 
                                          endtime=tp + aftP)
        ### Preprocess the data
        st.filter("lowpass", freq=4.0)
        st.resample(10)

        ### Only keep the station with 3 components
        if len(st) == 3:
            all_data.append(st)
            
    except:
        print(f"Failed to download data for station {station}")
        continue

---
Convert the stream data to numpy array.

The shape should be (N_station, 1500, 3), which the denoiser likes.

In [ ]:
data = np.array(all_data)[:,:,:npts]  # Stream --> Numpy Array
data = np.swapaxes(data, 1, 2)  # (N, 3, 1500) --> (N, 1500, 3)

print("The numpy array has the shape of", data.shape)

### Normalize the data for better visualization
data = (data - np.mean(data, axis=1, keepdims=True)) / (np.std(data, axis=1, keepdims=True) + 1e-10)

---
How does the raw data look like?

In [ ]:
plt.figure(figsize=(12, 6))

for i in range(len(data)):
    plt.text(10, i+0.2, station_list[i], fontsize=12, color='black')  # Station name
    plt.plot(data[i,:,0]/5+i, color='black')  # Z component waveform
    plt.vlines(preP/dt, -0.5, len(station_list)-0.5, color='black', linestyle='--')  # P time in theory
plt.ylim(-0.5, len(data)-0.5)
plt.xlabel('Time (s)')
plt.ylabel('Station index')
plt.title('Z component waveforms')

---
Save the raw data as a HDF5 file. 

This will be the input for the denoiser.

In [ ]:
with h5py.File('data.h5', 'w') as f:
    f.create_dataset('quake', data=data)

### Step 1. Denoise the earthquake waveforms in numpy array

Change the **config.ini** in your work folder first!

In [ ]:
# ! denote_predict  # Denoise the real but noisy data

print("\ndone! \ncheck the work directory for the denoised data and the sameple figure. ")

---
Save the results to a new HDF5 files.

This includes the denoised earthquake waveform and the separated noises.

In [ ]:
with h5py.File('separated_quake_and_noise.hdf5', 'r') as f:
    quake = f['quake'][:]
    noise = f['noise'][:]

We have selected a sample trace to plot the denoising performance (work folder).

Here, we can visualize all the denoised traces.


In [ ]:
plt.figure(figsize=(12, 6))
for i in range(len(quake)): 
    plt.text(10, i+0.2, station_list[i], fontsize=12, color='black')  # Station name
    plt.plot(quake[i,0,:]/5+i, color='black')  # Z component waveform
    plt.vlines(preP/dt, -0.5, len(station_list)-0.5, color='black', linestyle='--')  # P time in theory
plt.ylim(-0.5, len(quake)-0.5)
plt.xlabel('Time (s)')
plt.ylabel('Station index')
plt.title('Denoised earthquake signal')

---
How does the separated noises look like?

In [ ]:
plt.figure(figsize=(12, 6))
for i in range(len(noise)):
    plt.text(10, i+0.2, station_list[i], fontsize=12, color='black')
    plt.plot(noise[i,0,:]/5+i, color='black')
    plt.vlines(preP/dt, -0.5, len(station_list)-0.5, color='black', linestyle='--')
plt.ylim(-0.5, len(noise)-0.5)
plt.xlabel('Time (s)')
plt.ylabel('Station index')
plt.title('noise signal')

## Step 2. Retraining and testing the denoiser, if needed

In [ ]:
! denote_train

In [ ]:
! denote_test

print("Done testing the denoiser! Check the figure folder for results. ")